# 1. Generate ROI tiles

In [208]:
import os
from os.path import join, basename
from os import makedirs,cpu_count
from glob import glob
from concurrent.futures import ProcessPoolExecutor
import time
import rasterio
from rasterio.mask import mask
import fiona
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
import numpy as np

def gdal_polygonize(tif_path, gpkg_path):
    cmd = f'gdal_polygonize.py {tif_path} -b 1 -f "GPKG" {gpkg_path}'# OUTPUT DN'
    os.system(cmd)

def dissolve_features(input_gpkg,output_gpkg):
    #input_layer, 
    """
    Dissolve all features in a GeoPackage layer into a single geometry.

    :param input_gpkg: Path to the input GeoPackage file.
    :param input_layer: Name of the layer to dissolve.
    :param output_gpkg: Path to the output GeoPackage file with dissolved features.
    """
    # Load the layer from the GeoPackage
    gdf = gpd.read_file(input_gpkg)#, layer=input_layer)
    
    # Dissolve all features into a single geometry
    dissolved = gdf.dissolve()
    
    # Save the dissolved geometry to a new GeoPackage
    dissolved.to_file(output_gpkg, driver='GPKG') #layer='dissolved',
    print(f"Dissolved features saved to {output_gpkg}")

def buffer_features(input_gpkg,output_gpkg, buffer_distance):
    """
    Apply a buffer to the dissolved features in a GeoPackage layer.

    :param input_gpkg: Path to the input GeoPackage file with dissolved features.
    :param input_layer: Name of the layer to buffer.
    :param output_gpkg: Path to the output GeoPackage file with buffered features.
    :param buffer_distance: Distance for the buffer operation.
    """
    # Load the dissolved layer from the GeoPackage
    gdf = gpd.read_file(input_gpkg)#, layer=input_layer)
    
    # Apply the buffer operation
    buffered = gdf.buffer(buffer_distance)
    
    # Create a new GeoDataFrame to store the buffered geometry
    buffered_gdf = gpd.GeoDataFrame(geometry=buffered, crs=gdf.crs)
    
    # Save the buffered geometry to a new GeoPackage
    buffered_gdf.to_file(output_gpkg,driver='GPKG')# layer='buffered', 
    print(f"Buffered features saved to {output_gpkg}")

def save_to_gpkg(gdf, output_path):
    gdf.to_file(output_path, driver='GPKG')

def raster2vectorcutline(raster_path, gpkg_path):
    # Convert raster to polygons

    gpkg_path_int = gpkg_path.replace('.gpkg', '_P.gpkg')
    fb = gpkg_path_int.replace('.gpkg','_B.gpkg')
    gdal_polygonize(raster_path, gpkg_path_int)
    buffer_features(gpkg_path_int, fb, 0)
    dissolve_features(fb,gpkg_path)
    os.remove(gpkg_path_int)
    os.remove(fb)
 
    print("Process completed. Output saved to:", gpkg_path)

def clip_cultilinex(vpath,rpath, fopath):
    cmd = f'gdalwarp -cutline {vpath} -crop_to_cutline {rpath} {fopath}'
    #-dstalpha 
    os.system(cmd)

def clip_cultiline(vpath, rpath, fopath):
  # Open the vector file
  with fiona.open(vpath, "r") as shapefile:
      # Extract the geometry from the vector file
      shapes = [feature["geometry"] for feature in shapefile]

  # Open the raster file
  with rasterio.open(rpath) as src:
      # Clip the raster with the geometry
      out_image, out_transform = mask(src, shapes, crop=True)
      out_meta = src.meta.copy()

      # Update the metadata to reflect the new dimensions
      out_meta.update({
          "driver": "GTiff",
          "height": out_image.shape[1],
          "width": out_image.shape[2],
          "transform": out_transform
      })

      # Write the clipped raster to a new file
      with rasterio.open(fopath, "w", **out_meta) as dest:
          dest.write(out_image)

def get_out_params(tiles_dpath,bfile,src_dataset):
    dname = basename(bfile).replace('.gpkg','').split('_')[-1]
    dst_dpath = join(tiles_dpath,dname)
    makedirs(dst_dpath, exist_ok=True)
    dst_dataset = join(dst_dpath,basename(src_dataset))
    return dst_dataset


In [172]:
wdir = "/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/"
lidar_dpath = join(wdir, 'LiDAR')
tiles_dpath = join(wdir, 'TILES')

ds_dpath = "/media/ljp238/12TBWolf/ARCHIEVE/OUT/TILES12B/N13E103/"
vrt_files = ds_files = glob(f'{ds_dpath}/*.tif'); print(len(ds_files))

cpus = int(cpu_count() - 3)
basefiles = lidar_files = glob(f'{lidar_dpath}/*.tif')
lidar_files

18


['/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/LiDAR/Tonlesap_TSAP1.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/LiDAR/Tonlesap_TSAP2.tif']

In [209]:
def check_bands(rpath):
    with rasterio.open(rpath) as src:
        b = src.count
    print(f'{b}::{rpath}')

In [211]:
#for tfile in ds_files:check_bands(tfile)

generate gpkg from tifs

In [ ]:
gpkg_list = []
for j,fi in enumerate(basefiles):
    fo = fi.replace('.tif', '.gpkg')
    print(fi,'\n',fo)
    gpkg_list.append(fo)
   # raster2vectorcutline(fi,fo)

clip raster to cutiline gpkg

In [ ]:
gpkg_files = glob(f'{lidar_dpath}/*.gpkg')
gpkg_files,lidar_dpath

if __name__ == '__main__':
    ti = time.perf_counter()
    with ProcessPoolExecutor(cpus) as PPE:
        for i,vrt in enumerate(vrt_files):
            src_dataset = vrt
            #print(vrt)
            for j,bfile in enumerate(gpkg_files):
                dst_dataset = get_out_params(tiles_dpath,bfile,src_dataset)
                #clip_cultiline(src_dataset,bfile,dst_dataset)

                PPE.submit(
                   clip_cultilinex,bfile,src_dataset,dst_dataset
                )
 
               
    tf = time.perf_counter() - ti
    print(f'run.time : {tf/60}  min(s)')

# 2. Generate patches

In [214]:
import os
import itertools
import rasterio as rio
from rasterio import windows
from multiprocessing import Pool, cpu_count
from glob import glob

def get_tiles(ds, tile_width, tile_height, overlap):
  nols, nrows = ds.meta['width'], ds.meta['height']
  xstep = tile_width - overlap
  ystep = tile_height - overlap
  for x in range(0, nols, xstep):
      if x + tile_width > nols:
          x = nols - tile_width
      for y in range(0, nrows, ystep):
          if y + tile_height > nrows:
              y = nrows - tile_height
          window = windows.Window(x, y, tile_width, tile_height)
          transform = windows.transform(window, ds.transform)
          yield window, transform

def process_tile(args):
  window, transform, src_meta, in_path, out_path, output_filename = args
  metadata = src_meta.copy()
  metadata['transform'] = transform
  metadata['width'], metadata['height'] = window.width, window.height
  out_filepath = os.path.join(out_path, output_filename.format(window.col_off, window.row_off))
  
  with rio.open(in_path) as src:
      with rio.open(out_filepath, 'w', **metadata) as dst:
          dst.write(src.read(window=window))

def calculate_offset(width, height, tile_width, tile_height, stride_x, stride_y):
  X = [x for x in range(0, width, stride_x)]
  Y = [y for y in range(0, height, stride_y)]
  offsets = list(itertools.product(X, Y))
  return offsets

In [216]:
psize = 64
WDIR = "/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/"
tile_width = psize
tile_height = psize
overlap = 0
output_filename = 'tile_{}_{}.tif'
tilenames = os.listdir(WDIR)
tilenames

['TSAP1', 'TSAP2']

In [220]:
def gen_tilenames(WDIR, gpkg_files):
    tilenames = []
    for i in gpkg_files:
        bname = os.path.basename(i).split('/')[-1]
        bname = bname.split('_')[-1].replace('.gpkg', '')
        #print(bname)
        o = os.path.join(WDIR, bname)
        os.makedirs(o,exist_ok=True)
        tilenames.append(o)
    return tilenames
#tilenames = gen_tilenames(WDIR, gpkg_list)

In [ ]:
Stile = "N13E103"
for tilename in tilenames:
    tif_dpath = os.path.join(WDIR, tilename)
    patch_dpath = os.path.join(tif_dpath, f'P{psize}')
    os.makedirs(patch_dpath, exist_ok=True)
    tif_files = glob(f'{tif_dpath}/*.tif')
    for in_path in tif_files:
        out_path = os.path.join(patch_dpath, os.path.splitext(os.path.basename(in_path))[0])
        out_path = out_path.replace(f'{Stile}_','')
        print(out_path)
        os.makedirs(out_path, exist_ok=True)

        with rio.open(in_path) as src:
            src_meta = src.meta.copy()
            width, height = src_meta['width'], src_meta['height']
            stride_x = tile_width - overlap
            stride_y = tile_height - overlap
            offsets = calculate_offset(width, height, tile_width, tile_height, stride_x, stride_y)
            tiles = [(windows.Window(x, y, tile_width, tile_height), windows.transform(windows.Window(x, y, tile_width, tile_height), src.transform)) for x, y in offsets]

        args = [(window, transform, src_meta, in_path, out_path, output_filename) for window, transform in tiles]

        with Pool(processes=cpu_count()) as pool:
            pool.map(process_tile, args)

# 3. Generate df paths with nulls

In [222]:
import os
import pandas as pd
import numpy as np
import rasterio

def create_tif_dataframe(base_path):
  # Dictionary to hold lists of full paths to TIFF files for each subdirectory
  data = {}

  # Walk through the directory
  for root, dirs, files in os.walk(base_path):
      # Get the relative path of the current directory
      relative_path = os.path.relpath(root, base_path)

      # Initialize a list for the current directory if not already present
      if relative_path not in data:
          data[relative_path] = []

      # Filter and add full paths of TIFF files to the list
      for file in files:
          if file.endswith('.tif') or file.endswith('.tiff'):
              full_path = os.path.join(root, file)
              data[relative_path].append(full_path)

  # Create a DataFrame from the dictionary
  df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

  return df


def calculate_nan_ratio(file_path):
  if pd.isna(file_path):
      return np.nan

  with rasterio.open(file_path) as src:
      data = src.read(1)
      # Set nodata values to NaN
      nodata_value = src.nodata
      if nodata_value is not None:
          data[data == nodata_value] = np.nan

      # Set values < -30 and > 700 to NaN
      data[data < -30] = np.nan
      data[data > 700] = np.nan

      # Calculate the ratio of NaN values to total pixels
      nan_ratio = np.isnan(data).sum() / data.size

  return nan_ratio

from concurrent.futures import ThreadPoolExecutor
def calculate_nan_ratios_parallel(file_paths):
  with ThreadPoolExecutor() as executor:
      nan_ratios = list(executor.map(calculate_nan_ratio, file_paths))
  return nan_ratios

In [223]:
VARNAMES = ['cdem_DEM', 'cdem_WBM', 'edem_LCM', 'edem_W84', 'EGM08', 'EGM96',
       'label_thresh_ldem', 'label_thresh_pdem', 'multi_ALDTM', 'multi_ESAWC',
       'NegroAOIDTM', 'SENTINEL1', 'tdem_COM', 'tdem_DEM', 'tdem_DEM_M',
       'tdem_DEM_M_BIN', 'tdem_HEM', 'tdem_WAM']

In [224]:
rio_names = os.listdir(WDIR)
rio_names,WDIR

(['TSAP1', 'TSAP2'], '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/')

In [ ]:
dfpaths_list = []
for i in range(len(rio_names)):
    base_path = os.path.join(WDIR, rio_names[i],'P64')
    dfpaths = create_tif_dataframe(base_path)
    dfpaths = dfpaths[VARNAMES]
    fpath = base_path.replace('P64', 'P64_dfpaths.csv')

    a_ratios = calculate_nan_ratios_parallel(dfpaths['multi_ALDTM'])
    dfpaths['multi_ALDTM_ratio'] = a_ratios

    b_ratios = calculate_nan_ratios_parallel(dfpaths['tdem_DEM_M'])
    dfpaths['tdem_DEM_M_ratio'] = b_ratios
    print(fpath)
    dfpaths.to_csv(fpath, index=False)
    dfpaths_list.append(fpath)

In [226]:
dfpath = pd.concat(map(pd.read_csv, dfpaths_list))

In [227]:
dfpaths.multi_ALDTM_ratio.value_counts()

multi_ALDTM_ratio
0.000000    23
0.015625     5
1.000000     3
0.006348     1
0.029541     1
0.839844     1
0.128906     1
0.468750     1
0.056396     1
0.002686     1
0.175293     1
0.179199     1
0.004150     1
0.001465     1
0.012207     1
0.018555     1
0.020508     1
0.034912     1
0.210693     1
0.047363     1
Name: count, dtype: int64

In [228]:
dfpath.multi_ALDTM_ratio.value_counts()

multi_ALDTM_ratio
0.000000    269
1.000000     69
0.015625      7
0.187500      3
0.921875      2
           ... 
0.027832      1
0.909180      1
0.046631      1
0.058594      1
0.047363      1
Name: count, Length: 95, dtype: int64

# 4. Training Workflow TAB 
- BinClass
- RegDI
- RegID

In [230]:
#VARNAMES

In [231]:
FCOLX = ['EGM08','EGM96','tdem_HEM','edem_W84'] # add later 'SENTINEL1',
FCOLY = ['label_thresh_ldem','multi_ALDTM']#'ZDIF']
FCOLA = FCOLX + FCOLY
DFPATHS = dfpath[FCOLA]

In [232]:
from sklearn.model_selection import train_test_split
from pyspatialml import Raster

def get_patch(df, idx):
    return df.loc[idx:idx,:]

def get_patch_paths(df, idx,cols=None):
    if cols is None:
        return df.loc[idx:idx,:].values[0].tolist()
    else:
        df = df[cols].copy()
        return df.loc[idx:idx,:].values[0].tolist()


def tif2df(patch_paths,cols):
    patch_paths = [i for i in patch_paths for j in cols if j in i]
    assert len(patch_paths) == len(cols), "Length unequal"
    ds = Raster(patch_paths)
    ds.names = cols
   # tb = ds.to_pandas()
    return  ds.to_pandas()

def data_split(df_paths, test_pct=0.05, valid_pct=0.10):
    train_valid_paths, test_paths = train_test_split(df_paths, test_size=test_pct, random_state=130222)
    # Split train+valid into train (85%) and valid (10%)
    train_paths, valid_paths = train_test_split(train_valid_paths, test_size=valid_pct, random_state=130222)
    # Reset indices
    train_paths = train_paths.reset_index(drop=True)
    valid_paths = valid_paths.reset_index(drop=True)
    test_paths = test_paths.reset_index(drop=True)

    return train_paths, valid_paths, test_paths

In [233]:
train_paths, valid_paths, test_paths =  data_split(DFPATHS.sample(frac=.1))
print(train_paths.shape, valid_paths.shape, test_paths.shape)

(36, 6) (5, 6) (3, 6)


In [234]:
patch_paths = get_patch_paths(valid_paths,1,FCOLA)
patch_paths

['/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/TSAP1/P64/EGM08/tile_1216_384.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/TSAP1/P64/EGM96/tile_1472_128.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/TSAP1/P64/tdem_HEM/tile_192_640.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/TSAP1/P64/edem_W84/tile_1472_192.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/TSAP1/P64/label_thresh_ldem/tile_1216_384.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/TSAP1/P64/multi_ALDTM/tile_128_320.tif']

In [235]:
# solve the two band problem ::: they should have just one 
# check the tiles where came from and more 
# all the above 

In [236]:
ds = Raster(patch_paths)
len(list(ds.names))

6

In [237]:

di = tif2df(patch_paths,FCOLA)

In [239]:
di['label_thresh_ldem'].value_counts()

label_thresh_ldem
1.0    4094
0.0       2
Name: count, dtype: int64